In [1]:
# imports
import torch
import tensorflow as tf
import numpy as np
import csv
from collections import defaultdict

from transformers import (TFAutoModelWithLMHead, AutoTokenizer, 
    TFTrainer, TFTrainingArguments, TFT5ForConditionalGeneration, T5Config)

from mockup import Argument
from transformers import AutoTokenizer, AutoModelWithLMHead


2021-08-23 12:55:17.784547: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-23 12:55:17.784569: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data_csv_filename = "data/Webis-argument-framing.csv"
file = open(data_csv_filename)
dreader = csv.DictReader(file, dialect='unix', delimiter=',', quotechar='\"')
    
args = []
topic_frame_mapping = defaultdict(lambda : set())

    
for row in dreader:
    arg = Argument(row['argument_id'], row['frame_id'], row['frame'], row['topic_id'], row['topic'], row['premise'],
                   row['stance'], row['conclusion'])
    topic_frame_mapping['']
    args.append(arg)


In [3]:
X_text = []
Y_text = []
# simplest X_text

for arg in args:
    X_text.append(arg.premise + arg.conclusion)
    Y_text.append(arg.frame)

In [4]:
print('input:', X_text[0],'\n\n','output:', Y_text[1])

input: U.S. Treasury Secretary Henry Paulson summarized the rationale for the bailout in testimony before Congress in September, 2008 - "We must...avoid a continuing series of financial institution failures and frozen credit markets that threaten American families' financial well-being, the viability of businesses both small and large, and the very health of our economy."Not passing $700b bailout risks sending economy into major recession 

 output: economics


In [5]:
# setup t5
# this uses 'PreTrainedTokenizerFast' and 't5.TFT5ForConditionalGeneration' objects.
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = TFAutoModelWithLMHead.from_pretrained("t5-small")
print(tokenizer,model)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/home/olli/gits/T5-frame-identification/env/lib/python3.8/site-packages/transformers/models/auto/modeling_tf_auto.py:589: FutureWarning: The class `TFAutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `TFAutoModelForCausalLM` for causal language models, `TFAutoModelForMaskedLM` for masked language models and `TFAutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

2021-08-23 12:56:25.945155: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-23 12:56:25.945178: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-23 12:56:25.945196: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lucy): /proc/driver/nvidia/version does not exist
2021-08-23 12:56:25.945539: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-23 12:56:25.960819: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but thi

PreTrainedTokenizerFast(name_or_path='t5-base', vocab_size=32100, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45

In [6]:
# test t5
inputs = tokenizer(X_text[0],return_tensors='tf').input_ids
labels = tokenizer(Y_text[0],return_tensors='tf').input_ids
outputs = model.generate(inputs,max_length = 1000)
tokens = tokenizer.convert_ids_to_tokens(outputs[0])
tokenizer.convert_tokens_to_string(tokens)

'<pad><extra_id_0> in a statement before Congress in September, 2008 - "We must...avoid a continuing series of financial institution failures and frozen credit markets that threaten American families\' financial well-being, the viability of businesses both small and large," he said. "We must...avoid a continuing series of financial institution failures and frozen credit markets that threaten American families\' financial well-being," he said.</s>'

In [7]:
outs = model(inputs,labels=labels)
np.mean(outs.loss)

13.7461405

In [ ]:
X = tokenizer(X_text,return_tensors='tf',truncation=True,padding=True).input_ids
Y = tokenizer(Y_text,return_tensors='tf',truncation=True,padding=True).input_ids

print(X.shape)
print(Y.shape)
model.compile('adam',loss='sparse_categorical_crossentropy')
model.fit({'input_ids':X,'decoder_input_ids':X},Y,epochs=3)